[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_sdk/pdftools_sdk_multiple_img2_pdf.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDK/latest/pdftools_sdk-latest.tar.gz
%pip install ipython

# Convert multiple images to a PDF
Convert a list of images into a single PDF. Supported
image types are TIFF, JPEG, BMP, GIF, PNG, JBIG2, and
JPEG2000.

In [ ]:
import io
from pdftools_sdk.image import Document as ImageDocument, DocumentList as ImageDocumentList
from pdftools_sdk.image2_pdf import Converter
from pdftools_sdk.image2_pdf.profiles import Default

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
input_urls = [
    'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/ideas.tiff',
    'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/multipage.tif',
    'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/coffee.jpg',
    'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/connection.jpg'
]
input_image_paths = ['ideas.tiff', 'multipage.tif', 'coffee.jpg', 'connection.jpg']
for url, path in zip(input_urls, input_image_paths):
    prepare_file(url, path)
output_file_path = 'Combined.pdf'

In [ ]:
def images_to_pdf(input_image_paths: list[str], output_file_path: str):
    try:
        stream_list = []
        images = ImageDocumentList()

        # Open input images and store in list
        for input_image_path in input_image_paths:
            image_stream = io.FileIO(input_image_path, 'rb')
            stream_list.append(image_stream)
            images.append(ImageDocument.open(image_stream))

        # Create the profile that defines the conversion parameters.
        profile = Default()

        # Optionally the profile's parameters can be changed according to the 
        # requirements of your conversion process.

        # Create output stream
        with io.FileIO(output_file_path, 'wb+') as output_stream:
            converter = Converter()
            converter.convert_multiple(images, output_stream, profile)
    finally:
        if 'images' in locals():
            for image in images:
                image.__exit__(None, None, None)
        if 'stream_list' in locals(): 
            for stream in stream_list:
                stream.__exit__(None, None, None)

In [ ]:
try:
    # By default, a test license key is active. In this case, a watermark is added to the output. 
    # If you have a license key, please uncomment the following call and set the license key.
    # from pdftools_sdk.sdk import Sdk
    # Sdk.initialize("<PDFSDK,V1,MGAASQD6L2JMQHL54PK08RQX8GG4SS0M8DAHVPH0VMP3NB8R9DUK>")

    images_to_pdf(input_image_paths, output_file_path)

    print(f"Successfully created file {output_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")